### Import Libraries

In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

### Collect Data 
BTC/USDT
1 Year Historical Data
1. Futures price
2. Spot price
3. Funding rate

In [ ]:
# Binance API URLs
spot_url = "https://api.binance.com/api/v1/klines"
futures_url = "https://fapi.binance.com/fapi/v1/klines"
funding_rate_url = "https://fapi.binance.com/fapi/v1/fundingRate"

# Time range for 1 year (365 days)
end_time = datetime.now()
start_time = end_time - timedelta(days=365)

# Convert time to milliseconds
start_time_ms = int(time.mktime(start_time.timetuple()) * 1000)
end_time_ms = int(time.mktime(end_time.timetuple()) * 1000)

# Function to fetch historical data - for spot price and futures price
def fetch_data(url, symbol, interval, start_time, end_time, limit=1000):
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': limit
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

# Function to fetch funding rate data
def fetch_funding_rate(symbol, start_time, end_time):
    funding_rate_data = []
    # Funding rate is updated every 8 hours, fetch in chunks
    while start_time < end_time:
        params = {
            'symbol': symbol,
            'startTime': start_time,
            'endTime': min(start_time + 8 * 60 * 60 * 1000, end_time),  # 8 hours in ms
            'limit': 1000
        }
        response = requests.get(funding_rate_url, params=params)
        data = response.json()
        funding_rate_data.extend(data)
        start_time += 8 * 60 * 60 * 1000  # Move 8 hours forward
    return funding_rate_data

Get Futures and Spot Price Data

In [ ]:
# Fetch spot price data (OHLC for 8-hour interval)
spot_data = fetch_data(spot_url, 'BTCUSDT', '8h', start_time_ms, end_time_ms)
spot_df = pd.DataFrame(spot_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Fetch futures price data (OHLC for 8-hour interval)
futures_data = fetch_data(futures_url, 'BTCUSDT', '8h', start_time_ms, end_time_ms)
futures_df = pd.DataFrame(futures_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

In [21]:
# Convert timestamp to datetime
spot_df['timestamp'] = pd.to_datetime(spot_df['timestamp'], unit='ms')
futures_df['timestamp'] = pd.to_datetime(futures_df['timestamp'], unit='ms')

In [22]:
spot_df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2024-02-12 16:00:00,49917.80000000,50334.82000000,49372.22000000,49917.27000000,24675.04023000,1707782399999,1230954413.07840040,699209,12312.03484000,614345117.53386050,0
1,2024-02-13 00:00:00,49917.28000000,50368.61000000,49724.33000000,49983.09000000,10230.27828000,1707811199999,511558516.97429350,346956,5261.91345000,263148086.21383060,0
2,2024-02-13 08:00:00,49983.09000000,50247.00000000,48400.00000000,48727.47000000,28183.36700000,1707839999999,1392488190.51455020,818345,13836.41679000,683633139.28750190,0
3,2024-02-13 16:00:00,48727.47000000,49775.00000000,48300.95000000,49699.59000000,17137.92178000,1707868799999,840978662.57860340,576510,9311.46313000,457002287.17965800,0
4,2024-02-14 00:00:00,49699.60000000,49877.22000000,49225.01000000,49838.93000000,7205.37782000,1707897599999,357023932.42216920,330369,3470.38505000,172003072.20586820,0


In [24]:
futures_df.tail()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
995,2025-01-09 08:00:00,93176.50,94547.70,91747.00,94195.20,102810.293,1736438399999,9557608193.79620,1966408,49829.113,4633888074.36900,0
996,2025-01-09 16:00:00,94195.10,94644.80,91055.80,92505.90,99088.433,1736467199999,9175914071.77920,1905671,46214.789,4281388331.54340,0
997,2025-01-10 00:00:00,92506.00,94975.20,92395.90,94457.00,47137.689,1736495999999,4414395421.85170,1025917,23455.325,2197749968.89340,0
998,2025-01-10 08:00:00,94457.00,95239.90,92173.60,93704.00,134893.309,1736524799999,12663283848.20460,2326520,65856.448,6183114466.86990,0
999,2025-01-10 16:00:00,93704.00,95882.50,93151.50,94675.50,79911.956,1736553599999,7558804796.66060,1537507,42233.000,3995670597.08820,0


In [7]:
binance_data = pd.read_csv('binance_btc_usdt_1_year_data_8h.csv')

In [8]:
binance_data

,timestamp,close_spot,close_futures,fundingRate
0,2024-02-12 16:00:00,49917.27,49943.6,NaN
1,2024-02-13 00:00:00,49983.09,50019.9,NaN
2,2024-02-13 08:00:00,48727.47,48753.1,NaN
3,2024-02-13 16:00:00,49699.59,49696.4,NaN
4,2024-02-14 00:00:00,49838.93,49845.0,NaN
...,...,...,...,...
995,2025-01-09 08:00:00,94244.02,94195.2,NaN
996,2025-01-09 16:00:00,92552.49,92505.9,NaN
997,2025-01-10 00:00:00,94492.63,94457.0,NaN
998,2025-01-10 08:00:00,93730.92,93704.0,NaN


Get funding rate data

In [ ]:
# Fetch funding rate data (every 8 hours)
funding_rate_data = fetch_funding_rate('BTCUSDT', start_time_ms, end_time_ms)

In [ ]:
# Extract funding rate data (we only need the timestamp and funding rate)
funding_rate_df = pd.DataFrame(funding_rate_data, columns=['timestamp', 'fundingRate'])
funding_rate_df['timestamp'] = pd.to_datetime(funding_rate_df['timestamp'], unit='ms')

In [26]:
funding_rate_df

,timestamp,fundingRate
0,NaT,0.00010000
1,NaT,0.00010000
2,NaT,0.00010000
3,NaT,0.00010000
4,NaT,0.00010000
...,...,...
1090,NaT,0.00001802
1091,NaT,0.00009176
1092,NaT,0.00007457
1093,NaT,0.00004218


Merge the dataframes

In [ ]:
# Merge the dataframes based on the timestamp
merged_df = pd.merge(spot_df[['timestamp', 'close']], futures_df[['timestamp', 'close']], on='timestamp', suffixes=('_spot', '_futures'))
merged_df = pd.merge(merged_df, funding_rate_df[['timestamp', 'fundingRate']], on='timestamp', how='left')

Output CSV

In [ ]:
# Output to CSV
merged_df.to_csv('binance_btc_usdt_1_year_data_8h.csv', index=False)
print("Data saved to 'binance_btc_usdt_1_year_data_8h.csv'")